In [ ]:
# Import Libraries
import arcpy
import requests
import os
from zipfile import ZipFile

In [ ]:
# Define local directory
local_directory = r"E:\coursework\ARCGISI\LAB2\data"
# Create the local directory if it doesn't exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)
    
arcpy.env.workspace = r"E:\coursework\ARCGISI\LAB2\data"

## Part I

### Las ETL Pipeline

In [ ]:
# Downloads .LAS files from MN DNR
lidar_url = r'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-05.las'
response = requests.get(lidar_url)
file_name = os.path.join(local_directory, lidar_url.split("/")[-1])

with open(file_name, "wb") as f:
    f.write(response.content)

In [ ]:
# Converts the .LAS file into a DEM and Save to disk
arcpy.conversion.LasDatasetToRaster("4342-12-05.las", r"4342-12-05DEM.tif")

In [ ]:
# Converts the .LAS file into a TIN and Save to disk
arcpy.ddd.LasDatasetToTin("4342-12-05.las", r"4342-12-05TIN", "WINDOW_SIZE","CLOSEST_TO_MEAN")

In [ ]:
# Export as pdf
aprx = arcpy.mp.ArcGISProject("CURRENT")
lyt = aprx.listLayouts()[0]
lyt.exportToPDF(r"4342-12-05DEM.pdf", resolution=300)

lyt = aprx.listLayouts()[1]
lyt.exportToPDF(r"4342-12-05TIN.pdf", resolution=300)

### Bil ETL Pipeline

In [ ]:
# Downloads the annual 30-Year Normals .bil files for precipitation from PRISM 
bil_url = r'https://ftp.prism.oregonstate.edu/normals_4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'
response = requests.get(bil_url)
file_name = os.path.join(local_directory, bil_url.split("/")[-1])

with open(file_name, "wb") as f:
    f.write(response.content)
    
with ZipFile(file_name, "r") as zip_ref:
    zip_ref.extractall(local_directory)

In [ ]:
# Converts the data into a spacetime cube
sr = arcpy.SpatialReference(4326)
arcpy.env.workspace = r"E:\coursework\ARCGISI\LAB2\Default.gdb"
# Create mosaic
arcpy.management.CreateMosaicDataset(arcpy.env.workspace, "mosaic", sr)

In [ ]:
# Add Rasters to Mosaic
for i in ['01','02','03','04','05','06','07','08','09','10','11','12']:
    path = ["E:\coursework\ARCGISI\LAB2\data\PRISM_ppt_30yr_normal_4kmM4_", i,"_bil.bil"]
    input_file = ''.join(path)
    print(input_file)
    arcpy.management.AddRastersToMosaicDataset("mosaic", "Raster Dataset", input_file)

In [ ]:
# Create variable and timestamp
arcpy.management.CalculateField(r"mosaic\Footprint", "Variable", '"Precipitation"',"PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(r"mosaic\Footprint", "Timestamp", 'DateAdd(Date(2020, 0, 1), $feature.OBJECTID-1, "month")', "ARCADE", '', "DATE", "NO_ENFORCE_DOMAINS")


In [ ]:
# Build Multidimensional Info
arcpy.md.BuildMultidimensionalInfo("mosaic", "Variable", "Timestamp # #", None, "NO_DELETE_MULTIDIMENSIONAL_INFO")

In [ ]:
# Create Multidimensional Raster
arcpy.md.MakeMultidimensionalRasterLayer("mosaic", "mosaic_MultidimLayer","Precipitation")

In [ ]:
# Create Space Time Cube
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("mosaic_MultidimLayer", "prism_cube.nc", "SPATIAL_NEIGHBORS")

In [ ]:
# Export an animation of the timeseries 
arcpy.stpm.VisualizeSpaceTimeCube3D(r"E:\coursework\ARCGISI\LAB2\prism_cube.nc", "PRECIPITATION_NONE_SPATIAL_NEIGHBORS", "VALUE","cube_Visualize3d")